In [22]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [3]:
from transformers import pipeline
classifier = pipeline("text-classification", 
                      model="j-hartmann/emotion-english-distilroberta-base", 
                      top_k = None,
                      device = 0)
classifier("I love this!")

C:\Users\91939\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device set to use cuda:0


[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.008528688922524452},
  {'label': 'neutral', 'score': 0.0057645998895168304},
  {'label': 'anger', 'score': 0.004419785924255848},
  {'label': 'sadness', 'score': 0.002092393347993493},
  {'label': 'disgust', 'score': 0.0016119939973577857},
  {'label': 'fear', 'score': 0.0004138521908316761}]]

In [5]:
books = pd.read_csv('books_with_categories.csv')

In [6]:
books["description"][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [7]:
classifier(books["description"][0])

[[{'label': 'fear', 'score': 0.6548410654067993},
  {'label': 'neutral', 'score': 0.16985201835632324},
  {'label': 'sadness', 'score': 0.11640915274620056},
  {'label': 'surprise', 'score': 0.020700626075267792},
  {'label': 'disgust', 'score': 0.019100699573755264},
  {'label': 'joy', 'score': 0.015161334536969662},
  {'label': 'anger', 'score': 0.003935142885893583}]]

In [8]:
classifier(books["description"][0].split("."))

[[{'label': 'surprise', 'score': 0.7296031713485718},
  {'label': 'neutral', 'score': 0.14038558304309845},
  {'label': 'fear', 'score': 0.06816204637289047},
  {'label': 'joy', 'score': 0.04794229567050934},
  {'label': 'anger', 'score': 0.009156345389783382},
  {'label': 'disgust', 'score': 0.0026284733321517706},
  {'label': 'sadness', 'score': 0.0021221586503088474}],
 [{'label': 'neutral', 'score': 0.44937023520469666},
  {'label': 'disgust', 'score': 0.27359238266944885},
  {'label': 'joy', 'score': 0.10908287763595581},
  {'label': 'sadness', 'score': 0.09362705796957016},
  {'label': 'anger', 'score': 0.040478311479091644},
  {'label': 'surprise', 'score': 0.026970146223902702},
  {'label': 'fear', 'score': 0.006879040505737066}],
 [{'label': 'neutral', 'score': 0.6462156176567078},
  {'label': 'sadness', 'score': 0.24273361265659332},
  {'label': 'disgust', 'score': 0.04342273622751236},
  {'label': 'surprise', 'score': 0.028300514444708824},
  {'label': 'joy', 'score': 0.0142

In [10]:
sentences = books["description"][0].split(".")
predictions = classifier(sentences)

In [13]:
sentences[0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives'

In [14]:
predictions[0]

[{'label': 'surprise', 'score': 0.7296031713485718},
 {'label': 'neutral', 'score': 0.14038558304309845},
 {'label': 'fear', 'score': 0.06816204637289047},
 {'label': 'joy', 'score': 0.04794229567050934},
 {'label': 'anger', 'score': 0.009156345389783382},
 {'label': 'disgust', 'score': 0.0026284733321517706},
 {'label': 'sadness', 'score': 0.0021221586503088474}]

In [16]:
sorted(predictions[0], key= lambda x:x["label"])

[{'label': 'anger', 'score': 0.009156345389783382},
 {'label': 'disgust', 'score': 0.0026284733321517706},
 {'label': 'fear', 'score': 0.06816204637289047},
 {'label': 'joy', 'score': 0.04794229567050934},
 {'label': 'neutral', 'score': 0.14038558304309845},
 {'label': 'sadness', 'score': 0.0021221586503088474},
 {'label': 'surprise', 'score': 0.7296031713485718}]

In [18]:
emotions_labels = ['anger', 'disgust', 'fear',  'joy', 'neutral', 'sadness', 'surprise']
isbn = []
emotion_scores = {label : [] for label in emotions_labels}

def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label : [] for label in emotions_labels}
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key= lambda x:x["label"])
        for index, label in enumerate(emotions_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
    
    return {label: np.max(scores) for label , scores in per_emotion_scores.items()}

In [19]:
calculate_max_emotion_scores(predictions=predictions)

{'anger': np.float64(0.0641336739063263),
 'disgust': np.float64(0.27359238266944885),
 'fear': np.float64(0.928168535232544),
 'joy': np.float64(0.9327976107597351),
 'neutral': np.float64(0.6462156176567078),
 'sadness': np.float64(0.9671575427055359),
 'surprise': np.float64(0.7296031713485718)}

In [20]:
for i in range(10):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_score = calculate_max_emotion_scores(predictions)
    for label in emotions_labels:
        emotion_scores[label].append(max_score[label])

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [21]:
emotion_scores

{'anger': [np.float64(0.0641336739063263),
  np.float64(0.6126183867454529),
  np.float64(0.0641336739063263),
  np.float64(0.3514837324619293),
  np.float64(0.08141230791807175),
  np.float64(0.232224702835083),
  np.float64(0.5381843447685242),
  np.float64(0.0641336739063263),
  np.float64(0.3006705343723297),
  np.float64(0.0641336739063263)],
 'disgust': [np.float64(0.27359238266944885),
  np.float64(0.34828537702560425),
  np.float64(0.10400671511888504),
  np.float64(0.15072260797023773),
  np.float64(0.1844952404499054),
  np.float64(0.7271749377250671),
  np.float64(0.1558550000190735),
  np.float64(0.10400671511888504),
  np.float64(0.2794811427593231),
  np.float64(0.1779278963804245)],
 'fear': [np.float64(0.928168535232544),
  np.float64(0.9425278306007385),
  np.float64(0.9723208546638489),
  np.float64(0.3607063889503479),
  np.float64(0.09504327923059464),
  np.float64(0.051362842321395874),
  np.float64(0.7474278211593628),
  np.float64(0.40449532866477966),
  np.float

In [24]:
emotions_labels = ['anger', 'disgust', 'fear',  'joy', 'neutral', 'sadness', 'surprise']
isbn = []
emotion_scores = {label : [] for label in emotions_labels}

for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_score = calculate_max_emotion_scores(predictions)
    for label in emotions_labels:
        emotion_scores[label].append(max_score[label])

100%|██████████| 5197/5197 [03:03<00:00, 28.38it/s]


In [25]:
emotion_df = pd.DataFrame(emotion_scores)
emotion_df["isbn13"] = isbn

In [26]:
emotion_df.head()

,anger,disgust,fear,joy,neutral,sadness,surprise,isbn13
0,0.064134,0.273592,0.928169,0.932798,0.646216,0.967158,0.729603,9780002005883
1,0.612618,0.348285,0.942528,0.704422,0.887939,0.111690,0.252545,9780002261982
2,0.064134,0.104007,0.972321,0.767238,0.549477,0.111690,0.078766,9780006178736
3,0.351484,0.150723,0.360706,0.251881,0.732685,0.111690,0.078766,9780006280897
4,0.081412,0.184495,0.095043,0.040564,0.884390,0.475881,0.078766,9780006280934


In [27]:
books = pd.merge(books, emotion_df, on="isbn13")

In [28]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title_and_subtitle,tagged_description,simple_categories,anger,disgust,fear,joy,neutral,sadness,surprise
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.064134,0.273592,0.928169,0.932798,0.646216,0.967158,0.729603
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.612618,0.348285,0.942528,0.704422,0.887939,0.111690,0.252545
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,0.064134,0.104007,0.972321,0.767238,0.549477,0.111690,0.078766
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction,0.351484,0.150723,0.360706,0.251881,0.732685,0.111690,0.078766
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction,0.081412,0.184495,0.095043,0.040564,0.884390,0.475881,0.078766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,...,Mistaken Identity,9788172235222 On A Train Journey Home To North...,Fiction,0.148208,0.030643,0.919165,0.255171,0.853721,0.980877,0.030656
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,...,Journey to the East,9788173031014 This book tells the tale of a ma...,Nonfiction,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,0.227765
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,...,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,Fiction,0.009997,0.009929,0.339218,0.947779,0.375755,0.066685,0.057625
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,...,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction,0.064134,0.104007,0.459268,0.759456,0.951104,0.368111,0.078766


In [29]:
books.to_csv("books_with_emotions.csv", index=False)